<a href="https://colab.research.google.com/github/baroodb/code/blob/main/processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import concurrent.futures
data = np.random.randint(1, 10, size=(1000, 4, 4))
data.shape

def compute(mt):
  return np.max(mt)

def doSleep(id, duration):
  print('I am going to sleep for ', duration, ' seconds')
  time.sleep(duration)
  print('Slept for ', duration, ' seconds')
  return id 

durations = [1, 7, 6, 5, 10]
n_durations = [(id, d) for id, d in enumerate(durations)]

start = time.time()
results = []
for i in range(len(data)):
  results.append(compute(data[i]))
end = time.time()
print(f'Elapsed ', end - start, ' seconds')
print(results)

In [21]:
n_durations

[(0, 1), (1, 7), (2, 6), (3, 5), (4, 10)]

In [ ]:
def run():
  results = []
  with concurrent.futures.ProcessPoolExecutor() as executor:
    for result in executor.map(compute, data):
      results.append(result)
  
  return results

run()

In [5]:
def run_wrapper(p):
  return doSleep(*p)

def runIt():
  results = []
  with concurrent.futures.ProcessPoolExecutor() as exec:
    for _, result in zip(n_durations, exec.map(run_wrapper, n_durations)) :
      results.append(result)
  
  return results

runIt()

I am going to sleep for  1  seconds
I am going to sleep for  7  seconds
Slept for  1  seconds
I am going to sleep for  6  seconds
Slept for  7  seconds
I am going to sleep for  5  seconds
Slept for  6  seconds
I am going to sleep for  10  seconds
Slept for  5  seconds
Slept for  10  seconds


[0, 1, 2, 3, 4]

In [6]:
%load_ext autoreload
%autoreload 2

In [36]:
%%writefile mesh.py
import numpy as np
import concurrent.futures
import time 
import functools

video_poses = np.random.randint(1, 10, size=(1000, 4, 4))

#################################################
def doIt(position, ref_pose, video_pose):
  print('starting a process')
  time.sleep(1)
  computed = video_pose * video_pose * video_pose
  position = np.random.randint(20) * position * ref_pose
  return position


def callDoIt(data, position, ref_pose):
  res = []
  part = functools.partial(doIt, position, ref_pose)
  with concurrent.futures.ProcessPoolExecutor() as exec:
    for result in exec.map(part, data):
      res.append(result)
  
  return res
##################################################
def compute_3d(position, video_poses, direction, ref_pose, config):
  results = []
  
  for id, video_pose in enumerate(video_poses):
    computed = video_pose * video_pose * video_pose
    inclination = direction * (position+1)

    results.append(computed)
  
  return results, len(results)


def compute_3d2(position, video_poses, direction, ref_pose, config):
 
  
  data = [(id, video_pose) for id, video_pose in enumerate(video_poses)]
  print(data[0])
  
  results = callDoIt(video_poses, position, ref_pose)
  
  return results, len(results)


Overwriting mesh.py


In [30]:
np.random.randint(20)

13

In [37]:
from mesh import compute_3d2

position = 1
video_poses = np.random.randint(1, 10, size=(10, 4, 4))
direction = 'right'
ref_pose = 0.25
config = {
    'name': 'project', 
    'start': True
}
start = time.time()
rs = compute_3d2(position, video_poses, direction, ref_pose, config)
end = time.time()
print(f'Elapsed ', end - start, ' seconds')
print(rs)

(0, array([[5, 7, 9, 3],
       [1, 1, 9, 5],
       [8, 3, 6, 8],
       [6, 5, 2, 7]]))
starting a process
starting a process
starting a process
starting a process
starting a process
starting a process
starting a process
starting a process
starting a process
starting a process
Elapsed  5.056628704071045  seconds
([0.0, 0.0, 3.25, 3.25, 2.5, 2.5, 4.25, 4.25, 4.5, 4.5], 10)
